In [1]:
import pandas as pd

data = [
    ("Vardas1", 0, 165, 55),
    ("Vardas2", 1, 183, 95),
    ("Vardas3", 0, 171, 60),
    ("Vardas4", 1, 194, 102),
    ("Vardas5", 1, "?", 80),
    ("Vardas6", 1, 185, 90),
]

In [2]:
# pirmas
df = pd.DataFrame(data, columns=['vardas', 'lytis', 'ugis', 'svoris'])
filteredTable = df.drop(df[df['ugis'] == "?"].index)
# print(filteredTable)
# print('---- \n')

print(f"all average {filteredTable['ugis'].mean()}")
print(f"all median {filteredTable['ugis'].median()}")
print('\n ---- \n')

women = filteredTable[filteredTable['lytis'] == 0]
print(f"women average {women['ugis'].mean()}")
print(f"women median {women['ugis'].median()}")
print('\n ---- \n')

men = filteredTable[filteredTable['lytis'] == 1]
print(f"men average {men['ugis'].mean()}")
print(f"men median {men['ugis'].median()}")

all average 179.6
all median 183.0

 ---- 

women average 168.0
women median 168.0

 ---- 

men average 187.33333333333334
men median 185.0


In [3]:
#antra
from scipy.spatial import distance
import numpy as np

def norm(column):
    return (column - column.mean())/column.std()

def denorm(norm_var, column):
    return norm_var * column.std() + column.mean()

def metrikos_skaiciavimas(kintamieji, metrikos):
    return (1/np.sum(1/metrikos)*np.sum(kintamieji/metrikos))

norm_lytis = norm(df['lytis'])
# print(norm_lytis)

norm_ugis = norm(filteredTable['ugis'])
# print(norm_ugis)

norm_svoris = norm(df['svoris'])
# print(norm_svoris)



atstumas_e = np.sqrt((norm_lytis[4] - norm_lytis)**2+(norm_svoris[4]-norm_svoris)**2)
del atstumas_e[4]
# print(atstumas_e)

ugis_norm_e = metrikos_skaiciavimas(norm_ugis, atstumas_e)
ugis_e = denorm(ugis_norm_e, filteredTable['ugis'])
print(f"ugis euclidean - {ugis_e}")



atstumas_m = np.abs(norm_lytis[4] - norm_lytis)+np.abs(norm_svoris[4]-norm_svoris)
del atstumas_m[4]
# print(atstumas_m)

ugis_norm_m = metrikos_skaiciavimas(norm_ugis, atstumas_m)
ugis_m = denorm(ugis_norm_m, filteredTable['ugis'])
print(f"ugis manhattan - {ugis_m}")


atstumas_c = np.amax(
                np.stack(
                    (np.abs(norm_lytis[4] - norm_lytis), np.abs(norm_svoris[4]-norm_svoris)
                ), axis=-1), 
            axis=1).tolist()

del atstumas_c[4]
atstumas_c = pd.DataFrame(atstumas_c)[0]
# print(atstumas_c)

ugis_norm_c = metrikos_skaiciavimas(norm_ugis, atstumas_c)
ugis_c = denorm(ugis_norm_c, filteredTable['ugis'])
print(f"ugis Čebyšavo  - {ugis_c}")



ugis euclidean - 183.04910652784145
ugis manhattan - 183.81314372942035
ugis Čebyšavo  - 180.56052520951172


In [29]:
#trecias
filteredTable = filteredTable.astype({"ugis": np.int64})

avg_lyt = filteredTable['lytis'].mean()
avg_ugis = filteredTable['ugis'].mean()
avg_svoris = filteredTable['svoris'].mean()
print(f"average lyt {avg_lyt}")
print(f"average ugis {avg_ugis}")
print(f"average svoris {avg_svoris}")

corr = filteredTable.corr()
corr_lyt = corr['ugis'][0]
corr_svoris = corr['ugis'][2]
# print(corr_lyt)
# print(corr_svoris)

corr_atkurimas = avg_ugis + 1/(abs(corr_lyt) + abs(corr_svoris))

print(corr_atkurimas)


average lyt 0.6
average ugis 179.6
average svoris 80.4
180.13094425707627
